<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [ ]:
# Install spark

In [2]:
!pip install pyspark
!pip install findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 1.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 30.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=b5644b3209e6e335558bc612a4d657cc946c790a4dd02fc17ee03ef0499b128b
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/b7/8e/8f/ba5d017af5f502964eb1358e1d496a8519de1645936b01810e
Successfully built pyspark


In [ ]:
# Start session

In [3]:
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark.sql import SparkSession

# Create a Spark Context
sc = SparkContext()

# Create a Spark Session
spark = SparkSession.builder.appName("SparkMLOps").getOrCreate()

23/07/23 13:29:16 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
# Download The search term dataset from the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

In [5]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

--2023-07-23 13:29:27--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233457 (228K) [text/csv]
Saving to: ‘searchterms.csv.1’

searchterms.csv.1   100%[===================>] 227.99K  --.-KB/s    in 0.005s  

2023-07-23 13:29:27 (49.4 MB/s) - ‘searchterms.csv.1’ saved [233457/233457]



In [6]:
# Load the csv into a spark dataframe

In [7]:
df = spark.read.format("csv").option("header", "true").load("searchterms.csv")


In [ ]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)

In [13]:
print("Number of rows: ", df.count())
print("Number of columns: ", len(df.columns))


Number of rows:  10000
Number of columns:  4


In [ ]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)

In [14]:
df.show(5)


+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [ ]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

In [15]:
for column, dtype in df.dtypes:
    if column == "searchterm":
        print("Datatype of searchterm: ", dtype)


Datatype of searchterm:  string


In [ ]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

In [16]:
gaming_laptop_count = df.filter(df.searchterm == 'gaming laptop').count()
print("The term 'gaming laptop' was searched", gaming_laptop_count, "times")


The term 'gaming laptop' was searched 499 times


In [ ]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)

In [17]:
df.groupBy('searchterm').count().orderBy('count', ascending=False).show(5)

[Stage 14:=============================================>        (170 + 9) / 200]

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|mobile 6 inch| 2312|
|    mobile 5g| 2301|
|mobile latest| 1327|
|       laptop|  935|
|  tablet wifi|  896|
+-------------+-----+
only showing top 5 rows



In [ ]:
# The pretrained sales forecasting model is available at  the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

In [29]:
from pyspark.ml.regression import LinearRegressionModel
model = LinearRegressionModel.load('sales_prediction.model')

In [ ]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

In [30]:
from pyspark.ml.feature import VectorAssembler

def predict(year):
    # Convert the scalar year into a DataFrame
    data = [(year,)]
    df = spark.createDataFrame(data, ["year"])

    # Transform the year into a feature vector
    assembler = VectorAssembler(inputCols=["year"], outputCol="features")
    df = assembler.transform(df)

    # Make predictions using the loaded model
    predictions = model.transform(df)

    # Show the predictions
    predictions.show()

# Call the function
predict(2023)


+----+--------+------------------+
|year|features|        prediction|
+----+--------+------------------+
|2023|[2023.0]|175.16564294006457|
+----+--------+------------------+



23/07/23 14:01:11 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/07/23 14:01:11 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
